## Testing efficient methods for evaluation

* Using prequential, test-then-train and windowed evaluation. We show how the *_fast versions can performance. 
* The *_fast methods rely on the MOA implementation ```PrequentialEvaluation(stream, learner, ...)``` from ```moa.evaluation.EfficientEvaluationLoops```
* Performance may be affected due to data reading and where data sits (i.e. in memory or in disk).
  * When working with CSV files, the conversion process involves using NumpyStream, resulting in data being stored in memory as an Instances object (ARFF). As a result, the *_fast evaluation methods do not yield any performance improvement.
  * When ARFF files are employed, the data resides on disk, and each call to ```next_instance()``` necessitates reading a new row from the file. This can be perceived as a potential bottleneck, and future implementations could explore ways to enhance data access efficiency (like caching).

* [EXPERIMENTAL] Changed the behavior of *_evaluation() and *_evaluation_fast() such that if NumpyStream is used, then defaults to *_evaluation() and if not, *_evaluation_fast() is used. **Setting optimise=False disables this behavior.**

**Notebook last updated on 05/10/2023**

In [1]:
# Create the JVM and add the MOA jar to the classpath
from prepare_jpype import start_jpype
start_jpype()

MOA jar path location (config.ini): /Users/gomeshe/Dropbox/ciencia_computacao/dev/main-projects/MOABridge/jar/moa.jar
JVM Location (system): 
/Users/gomeshe/Library/Java/JavaVirtualMachines/openjdk-20.0.1/Contents/Home
JVM args: ['-Xmx8g', '-Xss10M']
Sucessfully started the JVM and added MOA jar to the class path


# File paths

In [2]:
hyper_arff_file_path = '/Users/gomeshe/Desktop/data/Hyper100k.arff'
covtfd_arff_file_path = '/Users/gomeshe/Desktop/data/covtNorm_fd2_numDummy50_conceptLen193669.arff'
# Stream with 580k instances and around 100 features (csv)
covtfd_csv_file_path = '/Users/gomeshe/Desktop/data/covtFD.csv'

## Real stream (arff) with 100k and ARF

```EvaluatePrequential -l (meta.AdaptiveRandomForest -l (ARFHoeffdingTree -k 6 -e 2000000 -g 50 -c 0.01) -s 30 -x (ADWINChangeDetector -a 0.001) -p (ADWINChangeDetector -a 0.01)) -s (ArffFileStream -f /Users/gomeshe/Desktop/data/Hyper100k.arff) -e BasicClassificationPerformanceEvaluator -f 1000000```

In [3]:
from stream import stream_from_file
from MOALearners import MOAClassifier
from evaluation import test_then_train_evaluation_fast
from ensembles import AdaptiveRandomForest

learner = AdaptiveRandomForest(ensemble_size=30)

stream = stream_from_file(path_to_csv_or_arff=hyper_arff_file_path, class_index=-1)

results = test_then_train_evaluation_fast(stream=stream, learner=learner, max_instances=None, sample_frequency=None)

display(results)
display(results['cumulative'].metrics_per_window())
print(results['cumulative'].accuracy())

{'learner': 'AdaptiveRandomForest',
 'cumulative': <evaluation.ClassificationEvaluator at 0x12b4dce50>,
 'wallclock': 50.533289194107056,
 'cpu_time': 61.535568999999995}

,classified instances,classifications correct (percent),Kappa Statistic (percent),Kappa Temporal Statistic (percent),Kappa M Statistic (percent)


87.51299999999999


In [4]:
from stream import stream_from_file
from MOALearners import MOAClassifier
from evaluation import test_then_train_evaluation
from ensembles import AdaptiveRandomForest

learner = AdaptiveRandomForest(ensemble_size=30)

stream = stream_from_file(path_to_csv_or_arff=hyper_arff_file_path, class_index=-1)

results = test_then_train_evaluation(stream=stream, learner=learner, max_instances=None, sample_frequency=None, optimise=False)

display(results)
display(results['cumulative'].metrics_per_window())
print(results['cumulative'].accuracy())

{'learner': 'AdaptiveRandomForest',
 'cumulative': <evaluation.ClassificationEvaluator at 0x105b0bd00>,
 'wallclock': 54.932321071624756,
 'cpu_time': 60.326436000000015}

,classified instances,classifications correct (percent),Kappa Statistic (percent),Kappa Temporal Statistic (percent),Kappa M Statistic (percent)


87.51299999999999


## Synthetic 10kk stream (RTG) with Naive Bayes

```EvaluatePrequential -l bayes.NaiveBayes -e BasicClassificationPerformanceEvaluator -i 10000000 -f 10000000```

In [5]:
from stream import RandomTreeGenerator
from MOALearners import MOAClassifier
from evaluation import test_then_train_evaluation_fast
from moa.classifiers.bayes import NaiveBayes

learner = MOAClassifier(moa_learner=NaiveBayes())
stream = RandomTreeGenerator()

results = test_then_train_evaluation_fast(stream=stream, learner=learner, max_instances=10000000, sample_frequency=None)

display(results)
display(results['cumulative'].metrics_per_window())
print(results['cumulative'].accuracy())

{'learner': 'NaiveBayes',
 'cumulative': <evaluation.ClassificationEvaluator at 0x103a46b80>,
 'wallclock': 12.506047010421753,
 'cpu_time': 13.003150000000005}

,classified instances,classifications correct (percent),Kappa Statistic (percent),Kappa Temporal Statistic (percent),Kappa M Statistic (percent)


73.64807


In [6]:
from stream import RandomTreeGenerator
from MOALearners import MOAClassifier
from evaluation import test_then_train_evaluation
from moa.classifiers.bayes import NaiveBayes

learner = MOAClassifier(moa_learner=NaiveBayes())
stream = RandomTreeGenerator()

results = test_then_train_evaluation(stream=stream, learner=learner, max_instances=10000000, sample_frequency=None, optimise=False)

display(results)
display(results['cumulative'].metrics_per_window())
print(results['cumulative'].accuracy())

{'learner': 'NaiveBayes',
 'cumulative': <evaluation.ClassificationEvaluator at 0x12b63bb80>,
 'wallclock': 149.2997808456421,
 'cpu_time': 149.184702}

,classified instances,classifications correct (percent),Kappa Statistic (percent),Kappa Temporal Statistic (percent),Kappa M Statistic (percent)


73.64807


## Real stream (covtFD) 580k+ instances with more than 100 features. 

using Naive Bayes

In [7]:
from stream import stream_from_file
from MOALearners import MOAClassifier
from evaluation import test_then_train_evaluation_fast
from moa.classifiers.bayes import NaiveBayes

learner = MOAClassifier(moa_learner=NaiveBayes())

stream = stream_from_file(path_to_csv_or_arff=covtfd_arff_file_path, class_index=-1)

results = test_then_train_evaluation_fast(stream=stream, learner=learner, max_instances=None, sample_frequency=None)

display(results)
display(results['cumulative'].metrics_per_window())
print(results['cumulative'].accuracy())

{'learner': 'NaiveBayes',
 'cumulative': <evaluation.ClassificationEvaluator at 0x103a0d9a0>,
 'wallclock': 40.007524728775024,
 'cpu_time': 42.744568000000015}

,classified instances,classifications correct (percent),Kappa Statistic (percent),Kappa Temporal Statistic (percent),Kappa M Statistic (percent)


52.24272862303812


In [8]:
from stream import stream_from_file
from MOALearners import MOAClassifier
from evaluation import test_then_train_evaluation
from moa.classifiers.bayes import NaiveBayes

learner = MOAClassifier(moa_learner=NaiveBayes())

stream = stream_from_file(path_to_csv_or_arff=covtfd_arff_file_path, class_index=-1)

results = test_then_train_evaluation(stream=stream, learner=learner, max_instances=None, sample_frequency=None, optimise=False)

display(results)
display(results['cumulative'].metrics_per_window())
print(results['cumulative'].accuracy())

{'learner': 'NaiveBayes',
 'cumulative': <evaluation.ClassificationEvaluator at 0x105b0bb80>,
 'wallclock': 52.40486693382263,
 'cpu_time': 52.16139499999997}

,classified instances,classifications correct (percent),Kappa Statistic (percent),Kappa Temporal Statistic (percent),Kappa M Statistic (percent)


52.24272862303812


# Testing the ```prequential_evaluation_fast``` 
## SRP100 + real dataset with 100k instances

```EvaluatePrequential -l (meta.StreamingRandomPatches -s 100) -s (ArffFileStream -f /Users/gomeshe/Dropbox/ciencia_computacao/lecturer/research/ssl_disagreement/datasets/RBFm/RBFm_100k.arff) -f 1000```

In [9]:
from stream import stream_from_file
from MOALearners import MOAClassifier
from moa.classifiers.meta import StreamingRandomPatches
from evaluation import prequential_evaluation_fast

cl_SRP = MOAClassifier(moa_learner=StreamingRandomPatches(), CLI='-s 100', random_seed=1)

stream = stream_from_file(path_to_csv_or_arff=hyper_arff_file_path, class_index=-1)

results = prequential_evaluation_fast(stream=stream, learner=cl_SRP, max_instances=None, window_size=10000)

display(results)
display(results['windowed'].metrics_per_window())
print(results['cumulative'].accuracy())

{'learner': 'StreamingRandomPatches',
 'cumulative': <evaluation.ClassificationEvaluator at 0x105b0ba00>,
 'windowed': <evaluation.ClassificationWindowedEvaluator at 0x105b0bca0>,
 'wallclock': 200.23040914535522,
 'cpu_time': 247.14606599999996}

,classified instances,classifications correct (percent),Kappa Statistic (percent),Kappa Temporal Statistic (percent),Kappa M Statistic (percent)
0,10000.0,87.48,74.961242,75.019952,74.808853
1,20000.0,88.91,77.796914,78.087335,77.362727
2,30000.0,88.73,77.468328,77.428400,77.585521
3,40000.0,89.73,79.457065,79.731597,79.298529
4,50000.0,89.88,79.758769,80.191818,79.514170
5,60000.0,90.13,80.252137,80.072683,79.836568
6,70000.0,89.99,79.981095,79.931836,79.980000
7,80000.0,90.29,80.582614,80.707332,80.252186
8,90000.0,90.36,80.718084,80.350591,80.819737
9,100000.0,90.08,80.154011,80.227227,79.951496


89.558


In [10]:
from stream import stream_from_file
from MOALearners import MOAClassifier
from moa.classifiers.meta import StreamingRandomPatches
from evaluation import prequential_evaluation

cl_SRP = MOAClassifier(moa_learner=StreamingRandomPatches(), CLI='-s 100', random_seed=1)

stream = stream_from_file(path_to_csv_or_arff=hyper_arff_file_path, class_index=-1)

results = prequential_evaluation(stream=stream, learner=cl_SRP, max_instances=None, window_size=10000, optimise=False)

display(results)
display(results['windowed'].metrics_per_window())
print(results['cumulative'].accuracy())

{'learner': 'StreamingRandomPatches',
 'cumulative': <evaluation.ClassificationEvaluator at 0x12b4fb550>,
 'windowed': <evaluation.ClassificationWindowedEvaluator at 0x105b0b340>,
 'wallclock': 210.6583309173584,
 'cpu_time': 251.697494}

,classified instances,classifications correct (percent),Kappa Statistic (percent),Kappa Temporal Statistic (percent),Kappa M Statistic (percent)
0,10000.0,87.48,74.961242,75.019952,74.808853
1,20000.0,88.91,77.796914,78.087335,77.362727
2,30000.0,88.73,77.468328,77.428400,77.585521
3,40000.0,89.73,79.457065,79.731597,79.298529
4,50000.0,89.88,79.758769,80.191818,79.514170
5,60000.0,90.13,80.252137,80.072683,79.836568
6,70000.0,89.99,79.981095,79.931836,79.980000
7,80000.0,90.29,80.582614,80.707332,80.252186
8,90000.0,90.36,80.718084,80.350591,80.819737
9,100000.0,90.08,80.154011,80.227227,79.951496


89.558


# Testing with a large CSV file

In [11]:
%%time
from stream import stream_from_file

# Loads the csv data to memory, it should take from 1 minute to 2 minutes. 
stream = stream_from_file(path_to_csv_or_arff=covtfd_csv_file_path, class_index=-1)

CPU times: user 5min 23s, sys: 7.02 s, total: 5min 30s
Wall time: 2min 1s


In [12]:
from MOALearners import MOAClassifier
from evaluation import test_then_train_evaluation
from moa.classifiers.bayes import NaiveBayes

learner = MOAClassifier(moa_learner=NaiveBayes())

results = test_then_train_evaluation_fast(stream=stream, learner=learner, max_instances=None, sample_frequency=None)

display(results)
display(results['cumulative'].metrics_per_window())
print(results['cumulative'].accuracy())

{'learner': 'NaiveBayes',
 'cumulative': <evaluation.ClassificationEvaluator at 0x12b66f0d0>,
 'wallclock': 40.6551251411438,
 'cpu_time': 40.63055299999996}

,classified instances,classifications correct (percent),Kappa Statistic (percent),Kappa Temporal Statistic (percent),Kappa M Statistic (percent)


57.97274061936866


In [13]:
from MOALearners import MOAClassifier
from evaluation import test_then_train_evaluation
from moa.classifiers.bayes import NaiveBayes

learner = MOAClassifier(moa_learner=NaiveBayes())

results = test_then_train_evaluation(stream=stream, learner=learner, max_instances=None, sample_frequency=None, optimise=False)

display(results)
display(results['cumulative'].metrics_per_window())
print(results['cumulative'].accuracy())

{'learner': 'NaiveBayes',
 'cumulative': <evaluation.ClassificationEvaluator at 0x105b0bd60>,
 'wallclock': 40.48186278343201,
 'cpu_time': 40.65406299999995}

,classified instances,classifications correct (percent),Kappa Statistic (percent),Kappa Temporal Statistic (percent),Kappa M Statistic (percent)


57.97274061936866


## Using prequential_evaluation_fast

In [14]:
from MOALearners import MOAClassifier
from evaluation import test_then_train_evaluation
from moa.classifiers.bayes import NaiveBayes

learner = MOAClassifier(moa_learner=NaiveBayes())

results = prequential_evaluation_fast(stream=stream, learner=learner, max_instances=None, window_size=None)

display(results)
display(results['cumulative'].metrics_per_window())
print(results['cumulative'].accuracy())

{'learner': 'NaiveBayes',
 'cumulative': <evaluation.ClassificationEvaluator at 0x12b66fe20>,
 'windowed': None,
 'wallclock': 41.434161901474,
 'cpu_time': 40.786838000000216}

,classified instances,classifications correct (percent),Kappa Statistic (percent),Kappa Temporal Statistic (percent),Kappa M Statistic (percent)


57.97274061936866


In [15]:
from MOALearners import MOAClassifier
from evaluation import test_then_train_evaluation
from moa.classifiers.bayes import NaiveBayes

learner = MOAClassifier(moa_learner=NaiveBayes())

results = prequential_evaluation(stream=stream, learner=learner, max_instances=None, window_size=None, optimise=False)

display(results)
display(results['cumulative'].metrics_per_window())
print(results['cumulative'].accuracy())

{'learner': 'NaiveBayes',
 'cumulative': <evaluation.ClassificationEvaluator at 0x105b0b910>,
 'windowed': None,
 'wallclock': 40.85849094390869,
 'cpu_time': 40.84165000000007}

,classified instances,classifications correct (percent),Kappa Statistic (percent),Kappa Temporal Statistic (percent),Kappa M Statistic (percent)


57.97274061936866
